##신입기수 교육: RNN Assignment
: text generation using RNN

: 데이터 이해/전처리를 읽고 난 후 모델 및 Text Generation 함수를 설계해보기

: 참고) text generation using RNN.ipynb파일, https://wikidocs.net/45101


**1) 데이터 이해/전처리**

In [1]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [4]:
# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# ArticlesApril2018.csv 파일 로드
df = pd.read_csv('/content/gdrive/MyDrive/ybigta_교육세션/09_RNN/과제/ArticlesApril2018.csv')

In [6]:
print('Number of columns: ', len(df.columns))
print(df.columns)

Number of columns:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [7]:
df['headline'].isnull().values.any()

False

In [8]:
# 모든 기사의 제목 하나의 리스트로 저장
headline = []
headline.extend(list(df.headline.values))
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [9]:
# headline 중 unknown 처리

print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [10]:
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.']

In [11]:
import re

# 데이터 전처리: 구두점 제거, lowercase()
def data_preprocessing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    return text

cnt=0
for i in headline:
    headline[cnt] = data_preprocessing(i)
    cnt+=1
headline[:10]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression',
 'commuter reprogramming',
 'ford changed leaders looking for a lift its still looking',
 'romney failed to win at utah convention but few believe hes doomed',
 'chain reaction',
 'he forced the vatican to investigate sex abuse now hes meeting with pope francis']

In [12]:
# 토큰화 및 단어 집합 크기 확인
t= Tokenizer()
t.fit_on_texts(headline)
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 3494


In [13]:
# 정수 인코딩 & 하나의 문장을 2 이상 길이의 훈련 데이터로 만들기"
sequences = []
for text in headline:
    for line in text.split('\n'):    # \n 기준으로 문장 토큰화
        encoded = t.texts_to_sequences([line])[0]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]   # 길이가 2 이상인 gram들 저장
            sequences.append(sequence)

cnt = 0
for i in sequences:
    if cnt < 11:
        cnt+=1
        print(i)

[99, 269]
[99, 269, 371]
[99, 269, 371, 1115]
[99, 269, 371, 1115, 582]
[99, 269, 371, 1115, 582, 52]
[99, 269, 371, 1115, 582, 52, 7]
[99, 269, 371, 1115, 582, 52, 7, 2]
[99, 269, 371, 1115, 582, 52, 7, 2, 372]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116]
[100, 3]


In [14]:
print(sequences)

[[99, 269], [99, 269, 371], [99, 269, 371, 1115], [99, 269, 371, 1115, 582], [99, 269, 371, 1115, 582, 52], [99, 269, 371, 1115, 582, 52, 7], [99, 269, 371, 1115, 582, 52, 7, 2], [99, 269, 371, 1115, 582, 52, 7, 2, 372], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], [100, 3], [100, 3, 1117], [100, 3, 1117, 2], [100, 3, 1117, 2, 14], [100, 3, 1117, 2, 14, 583], [100, 3, 1117, 2, 14, 583, 24], [100, 3, 1117, 2, 14, 583, 24, 1118], [100, 3, 1117, 2, 14, 583, 24, 1118, 373], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5, 1119], [1, 14], [1, 14, 1120], [1, 14, 1120, 1121], [15, 2], [15, 2, 584], [15, 2, 584, 4], [15, 2, 584, 4, 215], [15, 2, 584, 4, 215, 375], [15, 2, 584, 4, 215, 375, 1122], [15, 2, 584, 4, 215, 375, 1122, 2], [15, 2, 584, 4, 215, 375, 1122, 2, 376], [15, 2, 584, 4, 215, 375, 1122, 2, 376, 585], [8, 167], [8, 167, 2], [8,

In [15]:
# 빈도수 상위 단어 확인
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 576번 단어 : {}'.format(index_to_word[576]))
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 576번 단어 : victims
빈도수 상위 1번 단어 : the


In [16]:
# 가장 긴 샘플의 길이에 전체 샘플 길이 맞추기
max_len = max(len(l) for l in sequences) 
print('샘플 최대 길이 : {}'.format(max_len))

# 전체 샘플 길이를 max_len으로 패딩 작업
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print()
print('패딩 결과 확인')
print(sequences)

샘플 최대 길이 : 24

패딩 결과 확인
[[   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 [   0    0    0 ...  269  371 1115]
 ...
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]
 [   0    0    0 ...  115    2 1025]]


In [17]:
# 훈련 데이터에서 레이블 분리 (각 문장의 마지막 단어만 분리해서 y로)
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print('X 확인')
print(X)
print()
print('y 확인')
print(y)

# 레이블에 대한 one-hot encoding
y = to_categorical(y, num_classes = vocab_size)

print()
print('one-hot encoding : y 확인')
print(y)

X 확인
[[   0    0    0 ...    0    0   99]
 [   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 ...
 [   0    0    0 ...  170    8 3493]
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]]

y 확인
[ 269  371 1115 ...  115    2 1025]

one-hot encoding : y 확인
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**2) 모델 설계하기**

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [35]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 
model = Sequential()

model.add(Embedding(vocab_size, 32, input_length = max_len-1)) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model.add(SimpleRNN(32))   # 모델 및 은닉 상태 크기 설정 
model.add(Dense(vocab_size, activation = 'softmax')) #Dense Layer 및 활성화 함수 설정
          
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정 

model.fit(X,y,epochs = 130, verbose=2) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정 

Epoch 1/130
244/244 - 100s - loss: 7.6805 - accuracy: 0.0277 - 100s/epoch - 409ms/step
Epoch 2/130
244/244 - 2s - loss: 7.0977 - accuracy: 0.0319 - 2s/epoch - 9ms/step
Epoch 3/130
244/244 - 2s - loss: 6.9377 - accuracy: 0.0427 - 2s/epoch - 9ms/step
Epoch 4/130
244/244 - 2s - loss: 6.8278 - accuracy: 0.0461 - 2s/epoch - 9ms/step
Epoch 5/130
244/244 - 2s - loss: 6.6372 - accuracy: 0.0504 - 2s/epoch - 9ms/step
Epoch 6/130
244/244 - 3s - loss: 6.4984 - accuracy: 0.0570 - 3s/epoch - 12ms/step
Epoch 7/130
244/244 - 2s - loss: 6.3556 - accuracy: 0.0574 - 2s/epoch - 9ms/step
Epoch 8/130
244/244 - 2s - loss: 6.2143 - accuracy: 0.0629 - 2s/epoch - 9ms/step
Epoch 9/130
244/244 - 2s - loss: 6.0684 - accuracy: 0.0665 - 2s/epoch - 9ms/step
Epoch 10/130
244/244 - 2s - loss: 5.9256 - accuracy: 0.0733 - 2s/epoch - 9ms/step
Epoch 11/130
244/244 - 2s - loss: 5.7836 - accuracy: 0.0830 - 2s/epoch - 9ms/step
Epoch 12/130
244/244 - 2s - loss: 5.6432 - accuracy: 0.0920 - 2s/epoch - 9ms/step
Epoch 13/130
244/2

In [25]:
# 문장 생성 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # 데이터에 대한 패딩
        predict_x=model.predict(encoded, verbose=2) 
        classes_x=np.argmax(predict_x,axis=1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == classes_x: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [26]:
# Text Generation 결과 확인 
# 임의의 단어 'good'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'good', 10))

1/1 - 0s - 163ms/epoch - 163ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 18ms/epoch - 18ms/step
1/1 - 0s - 13ms/epoch - 13ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
1/1 - 0s - 16ms/epoch - 16ms/step
1/1 - 0s - 14ms/epoch - 14ms/step
good riddance mr speaker python for the cloud warriors were playing
